图像分割
===

CNN能够对图片进行分类，如果需要识别图片中特定部分的物体，就需要新的网络结构。通常CNN网络在卷积层之后会接上若干个全连接层，将卷积层产生的特征图映射成固定长度的特征向量。图像分割领域主要有5中网络，全卷机网络FCN，Unet，DeconvNet，SegNet，还有就是在Faster RCNN的基础上升级了的Mask RCNN网络

# 1.FCN(全卷积网络-Fully Convolutional Networks)
顾名思义，全卷积网络没有全连接层，他可以接收任意尺寸的输入图像，采用反卷积层对最后一个卷积层的特征图进行上采样，使它恢复到输入图像相同的尺寸，从而可以对每个像素都产生了一个预测，同时保留了原始输入图像的空间信息，最后在上采样的特征图上进行逐像素分类，最后逐个像素计算softmax分类的损失，相当于每一个像素对应一个训练样本。FCN包括三种类型的网络FCN-32s,FCN-16s和FCN-8s。

## 1.1.FCN-32s
Conv1+MaxPool=>$\frac{1}{2}$、Conv2+MaxPool=>$\frac{1}{4}$、Conv3+MaxPool$\widetilde{=>}\frac{1}{8}$、Conv4+MaxPool$\widetilde{=>}\frac{1}{16}$、Conv5+MaxPool$\widetilde{=>}\frac{1}{32}$、Conv6、Conv7得到原图的$\frac{1}{32}$，然后进行32倍上采样得到原图。$\widetilde{=>}$表示此特征图会保留

## 1.2.FCN-16s
在FCN-32s的基础上，将Conv7得到的最终结果进行2倍上采样，得到了原图的$\frac{1}{16}$，然后和Conv4+Pool之后的结果相加，得到原图的$\frac{1}{16}$，最后进行16倍的上采样得到原图

## 1.3.FCN-8s
在FCN-32s的基础上，将Conv7的结果进行4倍上采样，Conv4+Pool结果进行2倍上采样，以及Conv3+Pool，三个结果相加，得到原图的$\frac{1}{8}$，最后进行8倍上采样得到原图

最终的结果是8s的效果最好，32s的效果最差。FCN的优点是end-to-end, pixels-to-pixels,而且相比于传统的基于cnn做分割的网络更加高效,因为避免了由于使用像素块而带来的重复存储和计算卷积的问题;缺点同样明显，首先是训练比较麻烦,需要训练三次才能够得到FCN-8s,而且得到的结果还是不精细,对图像的细节不够敏感,这是因为在进行decode,也就是恢复原图像大小的过程时,输入上采样层的label map太稀疏,而且上采样过程就是一个简单的deconvolution，其次是对各个像素进行分类,没有考虑到像素之间的关系.忽略了在通常的基于像素分类的分割方法中使用的空间规整步骤,缺乏空间一致性

# 5. Mask RCNN
## 5.1.概述
Mask RCNN在Faster RCNN的基础上增加了图像分割的功能，相比较Faster RCNN，Mask RCNN有以下两点改动：
1. ROI Pooling Layer改成了ROIAlignment
2. 与bbox_pred和cls_score并排的，多了一个关于mask预测的分支。<br/>
![images](images/07.png)

## 5.2.FPN(Feature Pyramid Networks)特征金字塔网络
多数的物体检测算法都是只采用顶层特征做预测，但我们知道低层的特征语义信息比较少，但是目标位置准确；高层的特征语义信息比较丰富，但是目标位置比较粗略。另外虽然也有些算法采用多尺度特征融合的方式，但是一般是采用融合后的特征做预测，而FPN不一样的地方在于预测是在不同特征层独立进行的

### 5.2.1.思路
ConvNet的多个层提取出的不同尺度的feature maps，这形成一个pyramid hierarchy(金字塔结构，即底层大，高层小)，从上到下建立一个top-down pathway and lateral connections。(这个top-down的path就是用高层的特征来弥补底层的特征，从而得到高分辨率、强语义的预测，也就是弥补了上面不同尺度feature map之间通道不同存在的large segmantic gaps)。

### 5.2.2.Top-Down PathWay and Lateral Connections
- 高层feature上采样，采用最邻近上采样法(用这个方法是图简便)，spatial resolution放大2倍。处理后的feature map记为mapup
- 底层feature经过1×1的卷积layer,让feature的通道数和高层的feature map相同。处理后的feature map记为mapdown(论文中说的是reduce通道数，难道不是增加吗？)
- 将mapup和mapdown做像素相加(element-wise addition)，迭代直到产生finest resolution map
- 迭代后，在每个合并的feature map上使用3×3卷积得到最终map。（为了减少上采样过程中混叠效应）

CNN能够对图片进行分类，如果需要识别图片中特定部分的物体，就需要新的网络结构。通常CNN网络在卷积层之后会接上若干个全连接层，将卷积层产生的特征图映射成固定长度的特征向量。